# Library

In [1]:
import numpy as np
from progressbar import progressbar
from keras.datasets import mnist
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, accuracy_score,recall_score

Using TensorFlow backend.


In [ ]:
def load_mnist():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    x_train = x_train.reshape(-1, 784, 1) / 255.
    x_test = x_test.reshape(-1, 784, 1) / 255.
    y_train = to_categorical(y_train).reshape(-1,10,1)
    y_test = to_categorical(y_test).reshape(-1,10,1)
    return (x_train, y_train), (x_test, y_test)

# Matrix Functions

In [3]:
def generetor_matrix(rows,cols):
    return np.int_(np.random.rand(rows,cols)*10)

def add_matrix(m1,m2):
    return m1+m2
  
def subtract_matrix(m1,m2):
    return m1-m2

def multiply_matrix(m1,m2):
    return np.dot(m1,m2)

def generetor_matrix_nn(rows,cols):
    return (np.random.rand(rows,cols)*2)-1

def hadamard(m1,m2):
    return m1*m2

def escalar_multiply(m1,x):
    return m1*x

def trasnpose(m1):
    return m1.transpose()

# MPL Implementation

In [4]:
class NN:
    def __init__(self,i_nodes,h_nodes,o_nodes):
        self.i_nodes = i_nodes
        self.h_nodes = h_nodes
        self.o_nodes = o_nodes

        self.bias_ih = generetor_matrix_nn(self.h_nodes,1)
        self.bias_ho = generetor_matrix_nn(self.o_nodes,1)

        self.weigths_ih = generetor_matrix_nn(self.h_nodes,self.i_nodes)
        self.weigths_ho = generetor_matrix_nn(self.o_nodes,self.h_nodes)

        self.learning_rate = 0.01

    def signoid(self,x):
        return 1/(1+np.exp(-x))
  
    def dsignoid(self,x):
        return( x * (1-x))

    def train(self,input_,expected):
        #feedforward
        hidden = multiply_matrix(self.weigths_ih,input_)
        hidden = add_matrix(hidden,self.bias_ih)
        hidden = self.signoid(hidden)

        output = multiply_matrix(self.weigths_ho,hidden)
        output = add_matrix(output,self.bias_ho)
        output = self.signoid(output)

        #backpropagation

        #output to hidden
        output_error = subtract_matrix(expected,output)
        d_output = self.dsignoid(output)
        hidden_t = trasnpose(hidden)

        gradient = hadamard(d_output,output_error)
        gradient = escalar_multiply(gradient,self.learning_rate)
        #ajust Bias 0 to H
        self.bias_ho = add_matrix(self.bias_ho,gradient)

        weigths_ho_deltas = multiply_matrix(gradient,hidden_t)
        self.weigths_ho = add_matrix(self.weigths_ho,weigths_ho_deltas)


        #hidden to input
        weigths_ho_t = trasnpose(self.weigths_ho)
        hidden_error = multiply_matrix(weigths_ho_t,output_error)
        d_hidden = self.dsignoid(hidden)
        input_t = trasnpose(input_)

        gradient_h = hadamard(hidden_error,d_hidden)
        gradient_h = escalar_multiply(gradient_h,self.learning_rate)
        #ajust Bias H to I
        self.bias_ih = add_matrix(self.bias_ih,gradient_h)
        weigths_ih_deltas = multiply_matrix(gradient_h,input_t)
        self.weigths_ih = add_matrix(self.weigths_ih,weigths_ih_deltas)

    def predict(self,input_,expected):
        hidden = multiply_matrix(self.weigths_ih,input_)
        hidden = add_matrix(hidden,self.bias_ih)
        hidden = self.signoid(hidden)

        output = multiply_matrix(self.weigths_ho,hidden)
        output = add_matrix(output,self.bias_ho)
        output = self.signoid(output)
        
        return output.argmax()


In [5]:
nn = NN(784,400,10)
(x_train, y_train), (x_test, y_test) = load_mnist()

# Training

In [7]:
y_pred = []
y_true = []

while(1):
    for i in progressbar(range(1)):
        for j in progressbar(range(len(x_train))):
            nn.train(x_train[j],y_train[j])
    break

100% (60000 of 60000) |##################| Elapsed Time: 0:03:27 Time:  0:03:27
100% (1 of 1) |##########################| Elapsed Time: 0:03:27 Time:  0:03:27
100% (10000 of 10000) |##################| Elapsed Time: 0:00:01 Time:  0:00:01


# Prediction

In [ ]:
for i in progressbar(range(len(x_test))):
    y_pred.append(nn.predict(x_test[i],y_test[i]))
    y_true.append(y_test[i].argmax())

# Validation

In [16]:
cm = confusion_matrix(y_pred,y_true)
acc = accuracy_score(y_pred,y_true)
recall = recall_score(y_pred,y_true,average="macro")

In [20]:
print(cm)
print("Accuracy:",acc)
print("Recal:",recall)

[[ 963    0   13    6    2    8   15    2    3    3]
 [   0 1115    3    1    1    2    4   11    2    9]
 [   2    2  955   15    4    2    3   17    6    1]
 [   0    4   11  937    0   17    0    5   16    4]
 [   0    1   11    0  908    4    5   13   10   18]
 [   2    0    0   14    2  815    9    2   10    5]
 [   6    4    7    1   12   13  917    0    7    1]
 [   0    1    6    7    2    2    0  930    5    5]
 [   5    8   20   20    8   15    4    6  904    9]
 [   2    0    6    9   43   14    1   42   11  954]]
Accuracy: 0.9398
Recal: 0.9400592307897068
